## API Mini Project

#### This exercise will require you to pull some data from the Quandl API. Qaundl is currently the most widely used aggregator of financial market data.
As a first step, you will need to register a free account on the http://www.quandl.com website.
After you register, you will be provided with a unique API key, that you should store:


After you register, you will be provided with a unique API key, that you should store:

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.
For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.
You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series
While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the requests package, which can be easily downloaded using pip or conda. You can find the documentation for the package here: http://docs.python-requests.org/en/master/
Finally, apart from the requests package, you are encouraged to not use any third party Python packages, such as pandas, and instead focus on what's available in the Python Standard Library (the collections module might come in handy: https://pymotw.com/3/collections/ ). Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists. You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html
Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [230]:
# First, import the relevant modules

import numpy as np
import pandas as pd
import json

import requests

In [215]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = ("https://www.quandl.com/api/v3/datasets/WIKI/FB/data.json?") + API Key
ds = requests.get(url)

In [216]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(type(ds.json()))

<class 'dict'>


In [ ]:
ds.json()

### Task 1:
Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [76]:
ds = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?& start_date=2017-01-01&end_date=2017-12-31&api_key=") + API Key

### Task 2:
Convert the returned JSON object into a Python dictionary.

In [91]:
j= ds.json()

# Creating a pandas DataFrame to check my answers
# json_data_cols = json_data["dataset_data"]["column_names"]
# json_data_rows = json_data["dataset_data"]["data"]
# df = pd.DataFrame(json_data_rows, columns=json_data_cols)
# df.head()

In [ ]:
json_data

In [129]:
# Creating a pandas DataFrame to check my answers
json_data_cols = json_data["dataset_data"]["column_names"]
json_data_rows = json_data["dataset_data"]["data"]
df = pd.DataFrame(json_data_rows, columns=json_data_cols)
df.head(2)

,Date,Open,High,Low,Close,Change,Traded Volume,Turnover,Last Price of the Day,Daily Traded Units,Daily Turnover
0,2017-12-29,51.76,51.94,51.45,51.76,NaN,34640.0,1792304.0,None,None,NaN
1,2017-12-28,51.65,51.82,51.43,51.60,NaN,40660.0,2099024.0,None,None,NaN


### Task 3:
Calculate what the highest and lowest opening prices were for the stock in this period.

   **a. By using the pandas DataFrame**

In [145]:
# Using the pandas DataFrame to check my answers
df_open_max = df[["Date","Open"]][df.Open == df.Open.max()]
df_open_min = df[["Date","Open"]][df.Open == df.Open.min()]
print("Highest opening price and its date","\n",df_open_max[["Date","Open"]],"\n")
print("Lowest opening price and its date","\n",df_open_min[["Date","Open"]])

Highest opening price and its date 
          Date   Open
9  2017-12-14  53.11 

Lowest opening price and its date 
             Date  Open
4214  2001-09-06   4.2


   **b. By using for loop solution**

In [167]:
json_data["dataset_data"]["data"][1][1]

51.65

In [154]:
# For loop solution
open_max = float("-inf")
open_min = float("inf")
 
for a in json_data["dataset_data"]["data"]:
     if a[1] != None:
        if a[1] > open_max:
            open_max = a[1]
        if a[1] < open_min:
            open_min = a[1]
print ("The highest opening price is", open_max,"\n","\n","The lowest opening price is", open_min)

The highest opening price is 53.11 
 
 The lowest opening price is 4.2


**c. By using List Comprehension**

In [157]:
open_list = [a[1] for a in json_data["dataset_data"]["data"] if a[1] != None]
open_max = max(open_list)
open_min = min(open_list)

print("The highest opening price in 2017 was ${0:.2f} per share.".format(round(open_max, 2)))
print("The lowest opening price in 2017 was ${0:.2f} per share.".format(round(open_min, 2)))

The highest opening price in 2017 was $53.11 per share.
The lowest opening price in 2017 was $4.20 per share.


**d- By using Dictionary Comprehension**

In [ ]:
json_data["dataset_data"]["data"]

In [161]:
open_dict = {a[0]: a[1] for a in json_data["dataset_data"]["data"] if a[1] != None}
open_max_date, open_max = max(open_dict.items(), key=lambda k: k[1])
open_min_date, open_min = min(open_dict.items(), key=lambda k: k[1])

print("The highest opening price in 2017 was ${0:.2f} per share and occured on {1}." \
      .format(round(open_max, 2), open_max_date))
print("The lowest opening price in 2017 was ${0:.2f} per share and oocurred on {1}." \
      .format(round(open_min, 2), open_min_date))

The highest opening price in 2017 was $53.11 per share and occured on 2017-12-14.
The lowest opening price in 2017 was $4.20 per share and oocurred on 2001-09-06.


## Task 4:
What was the largest change in any one day (based on High and Low price)?

**a- By using loop solution:**

In [ ]:
spread_max = float("inf")
b=[]

for a in json_data["dataset_data"]["data"]:
    b = a[2] - a[3]
    if b > spread_max:
        spread_max = b
print('The largest change in any one day is ', spread_max)

**b-By using List Comprehension**

In [ ]:
spread_max = max(row[2] - row[3] for row in json_data["dataset_data"]["data"])
            
print("The largest daily price spread in 2017 was {1}${0:.2f} per share." \
      .format(round(spread_max, 2), u'\u0394'))

**c-By using Dictionary Comprehension**

In [ ]:
spread_dict = {row[0]: row[2] - row[3] for row in json_data["dataset_data"]["data"]}
spread_max_date, spread_max = max(spread_dict.items(), key=lambda k: k[1])

print("The largest daily price spread in 2017 was {1}${0:.2f} per share and occurred on {2}." \
      .format(round(spread_max, 2), u'\u0394', spread_max_date))

In [183]:
#Using the pandas DataFrame to check my answers
df_spread_max = df[["Date","High","Low"]][(df.High - df.Low) == (df.High - df.Low).max()]
df_spread_max

,Date,High,Low
390,2016-06-24,34.26,29.87


## Task 5:
What was the largest change between any two days (based on Closing Price)?

In [222]:
# For loop solution
close_delta_max = float("-inf")
close_prev =  json_data["dataset_data"]["data"][0][4]
 
for row in json_data["dataset_data"]["data"][1:]:
    close = row[4]
    close_delta = abs(close_prev - close)
    close_prev = close
    if close_delta > close_delta_max:
        close_delta_max = close_delta
print ('The largest change between consecutive day closing prices in 2017 was','$', close_delta_max,'per share')

The largest change between consecutive day closing prices in 2017 was $ 7.0 per share


In [188]:
# List comprehension solution
close = [row[4] for row in json_data["dataset_data"]["data"]]
close_delta_max = max(abs(close_prev - close) for close_prev, close in zip(close, close[1:]))
 
print("The largest change between consecutive day closing prices in 2017 was {1}${0:.2f}" \
      " per share.".format(round(close_delta_max, 2), u'\u0394'))

The largest change between consecutive day closing prices in 2017 was Δ$7.00 per share.


In [189]:
# Dictionary comprehension solution for abstracting other significant data
close_delta_dict = {}
close_prev =  json_data["dataset_data"]["data"][0][4]

for row in json_data["dataset_data"]["data"][1:]:
    close = row[4]
    close_delta_dict[row[0]] = abs(close_prev - close)
    close_prev = close

close_delta_max_date, close_delta_max = max(close_delta_dict.items(), key=lambda k:k[1])

print("The largest change between consecutive day closing prices in 2017 was {1}${0:.2f}" \
      " per share and occurred on {2}.".format(round(close_delta_max, 2), u'\u0394', \
                                               close_delta_max_date))

The largest change between consecutive day closing prices in 2017 was Δ$7.00 per share and occurred on 2000-06-08.


In [190]:
# Using the pandas DataFrame to check my answers
id_list = df.index[df.Close.diff().abs() == df.Close.diff().abs().max()].tolist()
id_list.insert(0, id_list[0] - 1)
df_close_delta_max = df.loc[id_list,["Date","Close"]]
df_close_delta_max["Diff"] = df_close_delta_max.Close.diff().abs()
df_close_delta_max

,Date,Close,Diff
4538,2000-06-09,45.0,NaN
4539,2000-06-08,38.0,7.0


## Task 6:
What was the average daily trading volume during this year?

In [221]:
# For loop solution
vol_sum = 0
vol_count = 0
 
for row in json_data["dataset_data"]["data"]:
    vol_sum += row[6]
    vol_count += 1
    
vol_avg = vol_sum / vol_count
print('The average daily trading volume for 2017 was', vol_avg)

The average daily trading volume for 2017 was 47040.76172649196


In [193]:
# List comprehension solution
volume = [row[6] for row in json_data["dataset_data"]["data"]]
vol_avg = sum(volume) / len(volume)

print("The average daily traded volume for 2017 was {0:,} shares".format(round(vol_avg, 1)))

The average daily traded volume for 2017 was 47,040.8 shares


In [194]:
# Using the pandas DataFrame to check my answers
df_vol_avg = df["Traded Volume"].mean()
df_vol_avg

47040.76172649196

## Task 7:
What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [195]:
# List comprehension solution
volume = [row[6] for row in json_data["dataset_data"]["data"]]
vol_sorted = sorted(volume)

if len(volume) % 2 != 0:
    vol_median = vol_sorted[len(volume) // 2]
else:
    vol_median = (vol_sorted[len(volume) // 2 - 1] + vol_sorted[len(volume) // 2]) / 2

print("The median daily traded volume for 2017 was {0:,} shares".format(round(vol_median, 1)))

The median daily traded volume for 2017 was 34,960.0 shares


In [196]:
# Using the pandas DataFrame to check my answers
df_vol_median = df["Traded Volume"].median()
df_vol_median

34960.0